In [48]:
import pickle
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
import spacy
from gensim.models.ldamulticore import LdaMulticore
from gensim.test.utils import datapath

In [41]:
import multiprocessing
multiprocessing.cpu_count()

8

In [9]:
jstor_df = pd.read_feather("../data/large_files/jstor_df_v1.feather")

In [10]:
len(jstor_df)

14103

In [11]:
jstor_df.head(5)

,creator,datePublished,docType,doi,id,identifier,isPartOf,issueNumber,keyphrase,language,...,sequence,tdmCategory,title,url,volumeNumber,wordCount,docSubType,sourceCategory,subTitle,hasPartTitle
0,[],1959-10-01,article,10.1177/001452465907100107,ark://27927/phx66812gq6,"[{'name': 'doi', 'value': '10.1177/00145246590...",Expository Times,1,"[omnipotent reigneth, god omnipotent, silence,...",[eng],...,7.0,"[Religion - Theology, Religion - Spiritual bel...",In the Study,http://doi.org/10.1177/001452465907100107,71,7382,None,None,None,None
1,[Victor Paul Furnish],2009-01-01,chapter,10.1017/CBO9780511621321.003,ark://27927/pbd6fpf5fh,"[{'name': 'isbn', 'value': '9780511621321'}, {...",Jesus According to PaulJesus According to Paul,None,"[saying, books online, jesus tradition, pauls ...",[eng],...,5.0,"[History - Historical methodology, Religion - ...",3 Sayings of Jesus in Paul's Letters,https://doi.org/10.1017/CBO9780511621321.003,None,8577,None,None,None,None
2,[Leander E. Keck],2015-01-01,chapter,None,ark://27927/phw1kd8s300,[],Christ&#39;s First Theologian,None,"[pharisaism, sandmel genius, judaism, rabbinic...",[eng],...,8.0,"[History - Historical methodology, Religion - ...",3. The Quest for Pauls Pharisaism,None,None,5354,None,None,None,None
3,[LeAnn Snow Flesher],2009-02-01,article,10.1177/003463730910600105,ark://27927/phx64fptrwj,"[{'name': 'doi', 'value': '10.1177/00346373091...",Review & Expositor: An International Baptist J...,1,"[scofield, premillennial, premillennial dispen...",[eng],...,5.0,"[Religion - Theology, Religion - Spiritual bel...",The Historical Development of Premillennial Di...,http://doi.org/10.1177/003463730910600105,106,3614,None,None,None,None
4,[A. Daunton-Fear],1995-07-01,article,10.1177/0040571X9509800404,ark://27927/phx64k1x5c2,"[{'name': 'doi', 'value': '10.1177/0040571X950...",Theology,784,"[baptism, holy spirit, communion, infant bapti...",[eng],...,4.0,[Religion - Spiritual belief systems],Resisting the Tide Christian Initiation and Co...,http://doi.org/10.1177/0040571X9509800404,98,4323,None,None,None,None


In [12]:
# in english only?
jstor_df["language"].apply(lambda x: ("eng" in x) & (len(x)==1)).sum()

14096

In [13]:
article_ids_dict = pickle.load(open("../data/article_ids_dict.pickle", "rb"))

In [31]:
dictionary = pickle.load(open("../data/dictionary_main.pickle", "rb"))
len(dictionary)

44592

In [15]:
decades = []
for decade_n in range(0,10):
    decades.append("19{0}0,19{0}9".format(str(decade_n)))
decades += ["2000,2009", "2010,2019"]
decades

['1900,1909',
 '1910,1919',
 '1920,1929',
 '1930,1939',
 '1940,1949',
 '1950,1959',
 '1960,1969',
 '1970,1979',
 '1980,1989',
 '1990,1999',
 '2000,2009',
 '2010,2019']

In [16]:
def ids_from_colvals(df_name, col, matchstring):
    ids = eval('{0}[{0}["{1}"]{2}]'.format(df_name, col, matchstring))["id"].tolist()
    return ids

In [17]:
decade = decades[2]
ids = ids_from_colvals("jstor_df", "publicationYear", ".between({})".format(decade))
len(ids)

132

In [32]:
class MyCorpus:
    def __iter__(self):
        for id in ids:
            short_id_str = str(article_ids_dict[id])
            id_filepath = "../data/large_files/article_docs/{}.pickle".format(short_id_str)
            for doc in pickle.load(open(id_filepath, "rb")):
                yield doc

In [33]:
ids = jstor_df["id"].tolist()
corpus = MyCorpus()  # doesn't load the corpus into memory!
print(corpus)

In [34]:
dct_corpus = Dictionary.from_corpus(corpus, id2word=dictionary)

In [35]:
vocabulary = list(dct_corpus.values())

In [36]:
len(dct_corpus)

44592

In [43]:
%%time
lda = LdaMulticore(corpus, num_topics=10, id2word=dct_corpus)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scip

CPU times: user 3min 40s, sys: 1min 28s, total: 5min 8s
Wall time: 6min 12s


In [51]:
lda.save("../data/large_files/lda_global_v1")

In [46]:
lda.show_topics(num_topics=10,num_words=10,log=False, formatted=True)

[(0,
  '0.025*"J" + 0.024*"early" + 0.016*"pp" + 0.016*"et" + 0.014*"H" + 0.013*"social" + 0.013*"Israel" + 0.013*"W" + 0.012*"D" + 0.012*"G"'),
 (1,
  '0.042*"John" + 0.022*"text" + 0.020*"man" + 0.020*"book" + 0.019*"Roman" + 0.017*"Spirit" + 0.017*"theological" + 0.017*"Holy" + 0.016*"love" + 0.014*"interpretation"'),
 (2,
  '0.076*"I" + 0.050*"church" + 0.035*"Christ" + 0.027*"biblical" + 0.024*"people" + 0.018*"C" + 0.014*"chapter" + 0.013*"Catholic" + 0.013*"American" + 0.011*"religion"'),
 (3,
  '0.035*"University" + 0.033*"Press" + 0.033*"Jewish" + 0.024*"human" + 0.019*"time" + 0.018*"good" + 0.017*"M" + 0.015*"example" + 0.014*"Cambridge" + 0.014*"Jews"'),
 (4,
  '0.059*"A" + 0.020*"world" + 0.020*"way" + 0.019*"divine" + 0.018*"theology" + 0.017*"E" + 0.016*"second" + 0.013*"come" + 0.012*"Rom" + 0.012*"body"'),
 (5,
  '0.026*"See" + 0.023*"religious" + 0.018*"5" + 0.018*"like" + 0.018*"As" + 0.018*"Christians" + 0.015*"R" + 0.014*"On" + 0.014*"7" + 0.013*"law"'),
 (6,
  '0.